In [1]:
! pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma langchain_huggingface faiss-cpu
! pip install --upgrade --quiet transformers sentence_transformers accelerate chromadb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlinker, which is not installed.
cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires ptxcompiler, which is not installed.
cuml 24.4.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.9.0 requires keras-core, which is not installed.
keras-nlp 0.12.1 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.2 which is incompatible.
cud

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain_huggingface.llms import HuggingFacePipeline


2024-07-03 17:54:15.524160: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 17:54:15.524262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 17:54:15.685026: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
login(token = 'hf_Szv......')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
import transformers
model_id = "meta-llama/Llama-2-7b-chat-hf"

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
import torch
from torch import cuda, bfloat16

query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",
        return_full_text=False
)

In [7]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
rr = llm(prompt="Why does a democratic country need a Constitution?")

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [8]:
print(rr)



A Constitution is a fundamental law of a country that outlines the structure and powers of its government, as well as the rights and freedoms of its citizens. In a democratic country, the Constitution serves several important purposes:

1. Establishes the framework of government: The Constitution sets out the structure of the government, including the separation of powers between the legislative, executive, and judicial branches. This helps to ensure that no one branch of government has too much power and that the system of checks and balances is in place.
2. Defines the powers of government: The Constitution defines the specific powers of each branch of government and limits the powers of the federal government. This helps to prevent the government from abusing its power and ensuring that the rights of citizens are protected.
3. Protects individual rights and freedoms: The Constitution includes a bill of rights that outlines the fundamental rights and freedoms of citizens, such as f

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load and split PDF
loader = PyPDFLoader('/kaggle/input/myraggg/Social_and_Political_Life_3_Class_VIII.pdf')
documents = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(documents)
print(len(docs))

373


In [34]:
from langchain_chroma import Chroma
# db = FAISS.from_documents(docs, embeddings)
vectordb = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory="chroma_db2")

retriever=vectordb.as_retriever()

In [35]:
question = "Why does a democratic country need a Constitution?"
searchDocs = vectordb.similarity_search(question)
print(searchDocs[0].page_content)

5
The Indian Constitution
Today most countries in the world have a Constitution.
While all democratic countries are likely to have a
Constitution, it is not necessary that all countries that have
a Constitution are democratic. The Constitution serves
several purposes. First, it lays out certain ideals that form
the basis of the kind of country that we as citizens aspire to
live in. Or, put another way, a Constitution tells us what
the fundamental nature of our society is. A country is
usually made up of different communities of people who
share certain beliefs but may not necessarily agree on all
issues. A Constitution helps serve as a set of rules and
principles that all persons in a country can agree upon as
the basis of the way in which they want the country to be
governed. This includes not only the type of government
but also an agreement on certain ideals  that they all believe
the country should uphold.Why Does a Country Need a Constitution?


In [37]:
from langchain_core.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Context: {context}

Question: {question}

Answer:
"""

prompt = ChatPromptTemplate.from_template(template)
print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:\n"))]


In [38]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# print(qa_chain)

In [46]:
response = qa_chain.invoke("Why does a democratic country need a Constitution? Answer in bullet points")

In [47]:
print(response)


* A Constitution lays out certain ideals that form the basis of the kind of country that we as citizens aspire to live in.
* A Constitution helps serve as a set of rules and principles that all persons in a country can agree upon as the basis of the way in which they want the country to be governed.
* A Constitution helps to protect us against certain decisions that we might take that could have an adverse effect on the larger principles that the country believes in.

Note: The answer is based on the given context and may not be applicable to all democratic countries.


## Lets verfiy RAG model response. 
Response of RAG should be present in pdf. 

In [78]:
question = "Why does a democratic country need a Constitution?"
searchDocs = vectordb.similarity_search(question)
print(searchDocs[0].page_content)

5
The Indian Constitution
Today most countries in the world have a Constitution.
While all democratic countries are likely to have a
Constitution, it is not necessary that all countries that have
a Constitution are democratic. The Constitution serves
several purposes. First, it lays out certain ideals that form
the basis of the kind of country that we as citizens aspire to
live in. Or, put another way, a Constitution tells us what
the fundamental nature of our society is. A country is
usually made up of different communities of people who
share certain beliefs but may not necessarily agree on all
issues. A Constitution helps serve as a set of rules and
principles that all persons in a country can agree upon as
the basis of the way in which they want the country to be
governed. This includes not only the type of government
but also an agreement on certain ideals  that they all believe
the country should uphold.Why Does a Country Need a Constitution?


In [79]:
# ! pip install PyPDF2

In [86]:
import PyPDF2

def search_string_in_pdf(pdf_path, search_string):
    # Open the PDF file
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        total_pages = len(reader.pages)
        
        # Iterate through each page
        for page_number in range(total_pages):
            page = reader.pages[page_number]
            text = page.extract_text()
            text = text.replace("\n", " ")
            
            # Check if the search string is in the text of the page
            if search_string in text:
                print(f"Found '{search_string}' on page {page_number + 1}")

# Verfiy point1 in response
pdf_path = '/kaggle/input/myraggg/Social_and_Political_Life_3_Class_VIII.pdf'
search_string = 'kind of country that we as citizens aspire to live in'
search_string_in_pdf(pdf_path, search_string)

# Verfiy point2 in response
search_string = 'serve as a set of rules and principles that all persons in a country can agree upon'
search_string_in_pdf(pdf_path, search_string)

# Verfiy point3 in response
search_string = 'protect us against certain decisions that we might take that could have an adverse effect on the larger principles that the country believes in'
search_string_in_pdf(pdf_path, search_string)


Found 'kind of country that we as citizens aspire to live in' on page 15
Found 'serve as a set of rules and principles that all persons in a country can agree upon' on page 15
Found 'protect us against certain decisions that we might take that could have an adverse effect on the larger principles that the country believes in' on page 19


<div style="color:white;
            display:fill;
            border-radius:15px;
            background-color:skyblue;
            font-size:100%;
            font-family:Verdana;
            letter-spacing:1px">
    <h2 style='padding: 20px;
              color:black;
              text-align:center;'>
        IF YOU LIKED THE NOTEBOOK PLEASE DO UPVOTE!  
    </h2>
    </div>